### Survey List Creation.
This notebook is used to create the call list for the FPC Dispatcher Survey. Each day, the call list is created using multiple separate files. The data is cleaned, transformed, merged, and run through a prediction model in order to create a list of dispatchers to contact. The list is then automatically sent via email to the callers.

In [144]:
# Imports
import pandas as pd
import pandas as pd 
import os
import shutil
import win32com.client as win
from datetime import datetime
import numpy as np

# Model imports
import pickle
from sklearn.linear_model import LogisticRegression

pd.options.mode.chained_assignment = None

In [145]:
# Functions
def days_since_login(val):
    val = val.split("T")[0].split('-')
    day = int(val[2])
    month = int(val[1])
    year = int(val[0])
    td = datetime.today()
    login = datetime(year, month, day)
    delta = td - login
    return delta.days

def size_category(size):
    '''Transforms carrier size to a categorical varialbe based on idustry standards'''
    if size == 1:
        return 1
    elif size > 1 and size < 11:
        return 2
    elif size > 10 and size < 51:
        return 3
    elif size > 50 and size < 201:
        return 4
    elif size > 200:
        return 5
    else:
        return 0

#### File Upload
There are six total files that will be used to create the call list. These include:
- `EUS Data`: complete user database of all FPC users
- `CARMA List`: file used to determine if a user's carrier is active in CARMA along with some other carrier information
- `Survey Data`: file of all the data collected from the current survey
- `Old Survey Data`: file of all the data collected from the previous call survey
- `Carrier Size`: file used to map carriers with then number of trucks they own
- `Done Users`: stores all of the users that have been sent out in a list, regardless of if they were contacted

In [146]:
rename_map = {
'Carrier Email':'Carrier_Email',
'userLastLogin':'Last_Login', 
'mcNumber':'MC',
'usDotNumber':'DOT'
}
valid_states = ['LA (LA)',
 'UT (UT)',
 'AZ (AZ)',
 'MS (MS)',
 'AR (AR)',
 'FL (FL)',
 'MA (MA)',
 'AB (AB)',
 'IL (IL)',
 'Other',
 'MO (MO)',
 'ID (ID)',
 'NV (NV)',
 'NJ (NJ)',
 'TN (TN)',
 'WA (WA)',
 'KY (KY)',
 'ON (ON)',
 'VA (VA)',
 'MI (MI)',
 'OR (OR)',
 'OH (OH)',
 'IN (IN)',
 'NY (NY)',
 'MD (MD)',
 'GA (GA)',
 'CT (CT)',
 'MN (MN)',
 'OK (OK)',
 'CA (CA)',
 'SC (SC)',
 'KS (KS)',
 'PA (PA)',
 'AL (AL)',
 'DE (DE)',
 'IA (IA)',
 'WI (WI)',
 'CO (CO)',
 'TX (TX)',
 'NE (NE)',
 'NC (NC)']

In [147]:
download_path = 'C:\\Users\\u77366\\Downloads'
td = datetime.today().strftime('%m/%d/%y').replace('/', '')
downloads = os.listdir(download_path) # list of all downloads
csv_download = f'CsvExtract{td}' # identifying name of CARMA files
file_path_list = [f'{download_path}\\{f}' for f in downloads if csv_download in f] # list of all CARMA downloads

# Finding the "list" file from downloads based on file size
if len(file_path_list) == 2:
    maxi = None
    Roles = None
    clist = None
    for f in file_path_list:
        if maxi == None or os.path.getsize(f) > maxi:
            roles = f
            maxi = os.path.getsize(f)
    file_path_list.pop(file_path_list.index(roles))
    clist = file_path_list.pop(0)
elif len(file_path_list) == 1:
    clist = file_path_list[0]
else:
    print('CARMA downloads error')
    assert False

# Other file downloads
raw_df = (pd.read_csv(r'C:\Users\u77366\OneDrive - Schneider\My files\Dev_FPS_Data\FPS_Carrier_Listing_RAW.csv', usecols=['upn', 'userLastLogin', 'firstName', 'lastName', 'mobileNumber', 'mcNumber', 'usDotNumber', 'userApplicationDetails'], low_memory=False)
    .rename(columns=rename_map))       
size_map = (pd.read_csv('C:\\Users\\u77366\\OneDrive - Schneider\\Desktop\\Python\\Survey\\FMCSA_CENSUS1_2022Mar.txt', encoding='latin1', usecols=['DOT_NUMBER', 'NBR_POWER_UNIT'])
    .rename(columns={'DOT_NUMBER':'DOT', 'NBR_POWER_UNIT':'Size'})
    .fillna(0))
survey_df = (pd.read_excel(r'Stored_Files\survey_df_placeholder.xlsx')
            .rename(columns={'Carrier Email':'upn'}))
old_survey_df = (pd.read_excel(r'Stored_Files\FPC Carrier Survey - Intern 2022 Project Copy.xlsx')
            .rename(columns={'Carrier Email':'upn'}))
clist_df = (pd.read_csv(clist)
    .drop(columns = ['Ref: Common Carrier ID Failure', 'Ref: Common Carrier ID', 'Country'], axis=1)
    .rename(columns = {'Carrier MC #':'MC', 'USDOT #':'DOT', 'Ref: Carrier Sponsor':'Sponsored' }))
du_df = (pd.read_csv(r'Stored_Files\dudf.csv', encoding='latin-1'))

In [148]:
# Removing all users that weren't contacted from du_df
du_df = du_df[du_df.upn.isin(survey_df.upn.to_list())]

# Transforming raw_df
raw_df = (raw_df
                .query('Last_Login.notna()')
                .assign(
                        DOT = lambda raw_df:raw_df.DOT.astype('string').fillna('0'),
                        MC = lambda raw_df:raw_df.MC.astype('string').fillna('0'), 
                        Last_Login = lambda raw_df:raw_df.Last_Login.astype('string').fillna('0'))
                .assign(
                        Login_Window = lambda raw_df:raw_df.Last_Login.apply(days_since_login))
                .query('Login_Window > 7 & Login_Window < 100 & userApplicationDetails.str.contains("carrierDispatcher")')
                )

# Separating raw_df by MC and DOT and merging each with CARMA data
raw_mc = (raw_df
                .query('MC != "0" & MC.str.isnumeric()',  engine='python')
                .assign(
                        MC = lambda raw_df:raw_df.MC.astype(float).astype('int64').astype(str))
                )
mc_df = (pd.merge(raw_mc, clist_df, on='MC', how = 'inner')
                .query('upn.notna() & Status == "ACTIVE (ACTIVE)"')
                .drop(columns=['DOT_x', 'userApplicationDetails', 'Status'])
                .rename(columns={'DOT_y':'DOT'})
                )
raw_dot = (raw_df
                .query('MC == "0" & DOT != "0"')
                .assign(
                        DOT = lambda raw_df:raw_df.DOT.astype(float).astype('int64').astype(str))
                )
dot_df = (pd.merge(raw_dot, clist_df, on='DOT', how = 'inner')
                .query('upn.notna() & Status == "ACTIVE (ACTIVE)"')
                .drop(columns=['MC_x', 'userApplicationDetails', 'Status'])
                .rename(columns={'MC_y':'MC'})
                )

In [149]:
# Creating a df of all CARMA active dispatchers
master_list = pd.concat([mc_df,dot_df])
master_list = (master_list
                .dropna(subset = 'mobileNumber')
                .drop_duplicates()
                .drop_duplicates(subset='mobileNumber')
                .drop(columns=['firstName', 'lastName'])
                .query('DOT.notna()')
                .assign(
                        DOT = lambda master_list:master_list.DOT.astype(int), 
                        State = lambda master_list:master_list.State.apply(lambda x:x if x in valid_states else 'Other'),
                        Sponsored = lambda master_list:master_list.Sponsored.apply(lambda x:1 if pd.notnull(x) else 0)
                        )
                )
master_list = pd.merge(master_list, size_map, left_on='DOT', right_on='DOT', how = 'left') # adding carrier sizes
master_list['Size_Category'] = master_list.Size.apply(size_category) # binning sizes

In [150]:
# Removing already contacted users from the previous survey and other carriers
old_survey_df = old_survey_df[old_survey_df['How did the user respond to the call?'] != 'No answer (multiple attempts)'] # allowing us to still contact users that didn't answer

master_list = (master_list
                [~master_list.upn.isin(old_survey_df.upn)]
                [(~master_list['MC'].isin(['114457', '1131563', '168085', '166960'])) & (~master_list['Carrier'].isin(['WEST MOTOR FREIGHT OF PA', 'DART TRANSIT COMPANY']))] # removing special carriers
)

C:\Users\u77366\AppData\Local\Temp\ipykernel_18312\1733201256.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  master_list = (master_list


In [151]:
# Model preparation and execution
pred_list = master_list[['Size_Category', 'State', 'Sponsored', 'Login_Window']]
pred_list = pd.get_dummies(pred_list, columns=['Size_Category'], prefix = 'Size', drop_first=True)
pred_list = pd.get_dummies(pred_list, columns=['Sponsored'], prefix = 'Sponsored', drop_first=True)
pred_list = pd.get_dummies(pred_list, columns=['State'], drop_first=True)
log_model = pickle.load(open(r'C:\Users\u77366\OneDrive - Schneider\Desktop\Python\Survey\Survey_Model.sav', 'rb')) # loading model
preds = log_model.predict_proba(pred_list)
master_list.loc[:,'Prediction'] = preds[:, 1]
master_list.loc[:,'Prediction'] = master_list.Prediction.apply(lambda x:1 if x > 0.34 else 0) # setting probability threshold at prespecified value
final_list = master_list[master_list.Prediction == 1]

In [154]:
final_list.head(5)

,upn,Last_Login,mobileNumber,MC,Login_Window,Carrier,DOT,City,State,Sponsored,Size,Size_Category,Prediction
4,chris_gaerlan@comcast.net,2022-08-22T18:54:19.9480412Z,+19164794072,1084810,8,JCNC TRUCKING LLC,3380328,Sacramento,CA (CA),0,2.0,2,1
12,support@odentrucking.com,2022-08-10T14:07:51.8207864Z,+16094175486,1084087,20,ODEN TRUCKING,3378763,MAGNOLIA,NJ (NJ),0,1.0,1,1
20,hdrodop@msn.com,2022-08-19T13:16:30.4985468Z,+12252925950,666470,11,RODOP TRUCKING INC,1838750,LEVITTOWN,PA (PA),0,1.0,1,1
25,buzzrun@hotmail.com,2022-08-15T02:29:17.8289709Z,+18655911555,132136,15,BUZZ RUN TRANSPORT INC,3186937,KNOXVILLE,TN (TN),0,1.0,1,1
31,Jwtruckingca@gmail.com,2022-08-10T04:23:11.1766662Z,+15596768433,608438,20,JW TRUCKING,1653580,COARSEGOLD,CA (CA),0,1.0,1,1


The final list has been created containing only users that the model has predicted to be stimulated by a call. A sample of this list will be mailed out to the caller(s) each day. 

In [153]:
# Storing data and sending list to caller(s)
call_list = final_list.sample(100).drop(columns=['Prediction'])
with open(r'Stored_Files\Lists\Call_List.csv', 'w', newline='\n') as f: # storing call list
    f.write(call_list.to_csv(index=False))

du_df = pd.concat([du_df, call_list]) 
with open(r'Stored_Files\dudf.csv', 'w', newline='\n') as f: # storing all users sent out on a list
    f.write(du_df.to_csv(index=False))

outlook = win.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'x'
mail.Subject = f'Call List - {datetime.today().strftime("%m/%d/%y")}'
mail.Body = 'Attached is the FPC dispatcher call list for today. Please contact me with any questions. \n\nThanks,\n\nTrent'
mail.Attachments.Add(r'Stored_Files\Lists\Call_List.csv')
mail.send

com_error: (-2147352567, 'Exception occurred.', (4096, 'Microsoft Outlook', 'Path does not exist. Verify the path is correct.', None, 0, -2147024893), None)